# Profiling Code Run Times

## 1. Snippet Performance

When we want to compare the performance of small pieces of code to perform single tasks.

### 1.1 Using `time`

In [ ]:
from collections import Counter
from time import time
import numpy as np
import matplotlib.pyplot as plt

We have some integer IDs and the frequency of their occurrence within a dataset. We want to remove any integers with a frequency above a certain threshold.

In [ ]:
def generate_ids(p=.00005, size=1_000_000):
    """Generate an array of random integers with a geometric frequency distribution.
    
    Args:
        p: See `p` for `numpy.random.geometric`.
        size: Length of generated array.
        
    Returns:
        Array of integers.
    """
    idx = np.random.geometric(p, size) - 1
    x = range(np.max(idx))
    choices = np.random.choice(x, idx.shape[0])
    print("Number of unique IDs: {}".format(np.unique(idx).shape[0]))
    return choices[idx]

In [ ]:
unique_ids = generate_ids()
unique_ids_counter = Counter(unique_ids)
print("Most common IDs:", unique_ids_counter.most_common(20))

In [ ]:
fig, ax = plt.subplots()
ax.hist(
    unique_ids_counter.values(), 
    bins=100, 
    cumulative=True, 
    density="normed", 
    histtype="step", 
    linewidth=2
)
ax.set_xlabel("ID Count")
ax.set_ylabel("Normalised Cumulative Frequency")
ax.set_xlim(0, max(unique_ids_counter.values()));

In [ ]:
from operator import itemgetter

def filter_freq_map_filter(ids_counter, min_frequency=10):
    """Filters items from a `Counter` with frequencies below `min_frequency`. 
    
    Gets items from counter after applying `filter` operation.
    """
    return list(map(itemgetter(0), filter(lambda x: x[1] >= min_frequency, ids_counter.items())))

def filter_freq_iterate(ids_counter, min_frequency=10):
    """Filters items from a `Counter` with frequencies below `min_frequency`.
    
    Iterates through items in counter and drops keys with frequencies below
    `min_frequency`.
    """
    return [k for k, v in ids_counter.items() if v >= min_frequency]

In [ ]:
def time_function(function, **kwargs):
    start = time()
    function(**kwargs)
    end = time()
    total = end - start
    print(f"Took {total} seconds")

In [ ]:
print("Using map and filter...")
time_function(filter_freq_map_filter, **{'ids_counter': unique_ids_counter})

In [ ]:
print("Using list comprehension...")
time_function(filter_freq_iterate, **{'ids_counter': unique_ids_counter})

### 1.2 Using `timeit`

In [ ]:
import timeit

In [ ]:
t = timeit.Timer()

In [ ]:
t_i = timeit.repeat(
    stmt='filter_freq_iterate(unique_ids_counter)', 
    setup="from __main__ import filter_freq_iterate, unique_ids_counter",
    repeat=5,
    number=100
)
sorted(t_i)

In [ ]:
t_mf = timeit.repeat(
    stmt='filter_freq_map_filter(unique_ids_counter)', 
    setup="from __main__ import filter_freq_map_filter, unique_ids_counter",
    repeat=5,
    number=100
)
sorted(t_mf)

### 1.3 Using Magic `time` and `timeit`

In [ ]:
%%time
_ = filter_freq_map_filter(unique_ids_counter)

In [ ]:
%%timeit
_ = filter_freq_map_filter(unique_ids_counter)

In [ ]:
%%time
_ = filter_freq_iterate(unique_ids_counter)

In [ ]:
%%timeit
_ = filter_freq_iterate(unique_ids_counter)

### 1.4 Exercise

Write two or more different simple functions to complete one of the tasks below and time how long they take to execute on some dummy data. Were the results what you expected?

- Removing punctuation from a string
- Fetching items from a list based on their index location
- Finding the index of an item in a list
- Calculating the Jaccard similarity between sets of sets

### 1.5 Realistic Timing and Estimating Run times

When we process large datasets in a pipeline, we often need to estimate how long it might take. Long running processes are fragile to interruptions, slow down our work and can cost a lot more to run in terms of energy and money. 

Before running a piece of code on a full set of data, it's useful to prototype on a smaller sample and estimate the running time for the full set. This is good practice anyway, as you don't want to run a long pipeline on all of your data only to find you made a mistake that results in erroneous ouputs.

When testing your code on a sample to estimate run times or to compare performance, bear in mind:

- 📈 Run times are not always linear. For example, applying a function to larger batches of data and/or processing data with higher dimensions may have a quadratic or exponential effect on total running time. If you're unsure, test a few different parameters.
- 🏗 When testing how long it takes your code to run on a sample it can be tempting to just run it on the first N rows. In some cases however, the order of your data will have an impact. For example, you may have text data that increases in length with the number of rows, which would lead you to underestimate the total run time.
- 🔋 Some code involves overheads such as building indices, loading lookups or preparing data for parallel processing. Ideally these overheads are minimal but in some cases their runtimes can be comparable to the rest of your code. It's important to find the right trade off between invoking these overheads and the total amount of time your code takes to run.

## 2. Line Profiling

Usually, our code is more complex than a few snippets or one liners, particularly in long data processing or modelling pipelines, which are typically formed from multiple functions of varying complexity. If you time your code while prototyping, you might find that it takes too long to run and seek ways to speed it up. `line_profiler` helps you to identify which parts of your code are taking the longest and to think about strategies for speeding them up.

In [ ]:
!pip install line_profiler
%load_ext line_profiler

### 2.1 Filtering IDs... again

In our previous example, we filtered the IDs whose frequencies had already been counted.

```python
unique_ids_counter = Counter(unique_ids)
filtered = filter_freq_iterate(unique_ids_counter)

```

But, what if we don't recieve counts, but the raw IDs instead? In many real situations, this construction time will also need to be taken into account.

In [ ]:
def filter_id_freqs_iter(ids, min_frequency=10):
    """Filters items from a list with frequencies below `min_frequency`.
    
    Iterates through items in counter and drops keys with frequencies below
    `min_frequency`.
    """
    counts = Counter(ids)
    return [k for k, v in counts.items() if v >= min_frequency]

In [ ]:
%%time
_ = filter_id_freqs_iter(unique_ids)

This is much slower!

The only thing we have changed is that we're now counting the IDs inside the function. We would expect this to be the cause of the slow down, but let's take a look to see.

In [ ]:
%load_ext line_profiler

The line profiler extension allows us to profile code by timing how long it takes to run each line within a function.

```python
%lprun \            # Use the line profiler magic function
-f my_function \    # Pass in the name of the function you want to profile
my_function(a, b)   # Run the function on some inputs
```

In [ ]:
%lprun \
-f filter_id_freqs_iter \
filter_id_freqs_iter(unique_ids)

In [ ]:
def filter_id_freqs_np(ids, min_frequency=10):
    """Filters items from a list with frequencies below `min_frequency`
    
    Uses `np.unique` to calculate the counts of each item and then 
    subsets those above the threshold frequency from the unique values.
    """
    vals, counts = np.unique(ids, return_counts=True)
    return vals[counts >= min_frequency]

In [ ]:
%%time
_ = filter_id_freqs_np(unique_ids)

In [ ]:
%lprun -f filter_id_freqs_np filter_id_freqs_np(unique_ids)

### 2.2 Vectorisation sensation

Note: This example is taken from [Mortada Mehyar's blog](https://mortada.net/easily-profile-python-code-in-jupyter.html).

In [ ]:
import numpy as np
import pandas as pd

origin = {'lat': 34, 'lon': -120}

def generate_random_walk():
    np.random.seed(1)
    n = 100000
    changes = np.random.randn(n, 2) / 30
    changes[0] = [0, 0]
    trace = pd.DataFrame.from_records(changes, columns=['lat', 'lon']).cumsum()
    trace['lat'] += origin['lat']
    trace['lon'] += origin['lon']
    return trace

trace = generate_random_walk()

plt.plot(trace['lat'], trace['lon'])
plt.scatter(origin['lat'], origin['lon'], color='C1', s=100);

**Haversine Distance**
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Illustration_of_great-circle_distance.svg/1920px-Illustration_of_great-circle_distance.svg.png" alt="Haversine" width="200"/>

[Source: Wikipedia](https://en.wikipedia.org/wiki/Great-circle_distance)

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lat1, lon1, lat2, lon2):
    """Calculate the great circle distance between two points 
    on the earth (specified in lat/lon)
    """
    # convert to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    earth_radius = 6367
    distance_km = earth_radius * c
    return distance_km

def get_distances(trace, origin):
    distances = {}
    for i in trace.index:
        distances[i] = haversine(
            trace['lat'].loc[i], trace['lon'].loc[i], origin['lat'], origin['lon']
        )
    distances = pd.Series(distances)
    return distances

def get_farthest(trace, origin):
    distance = get_distances(trace, origin)
    max_idx = distance.argmax()
    return trace.loc[max_idx], distance.loc[max_idx]

In [ ]:
%%time
get_farthest(trace, origin)

In [ ]:
%lprun -f get_farthest get_farthest(trace, origin)

You can also use the line profiler to profile specific calls made within the function you are profiling. We do this by passing in the sub-function name under the `-f` flag. Here we will profile `get_distances` as it runs when we call `get_farthest` with our arguments.

In [ ]:
%lprun -f get_distances get_farthest(trace, origin)

In [ ]:
def haversine_vectorized(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth. Note that lat1/lon1/lat2/lon2 can either be
    scalars or numpy arrays.
    """
    # convert to radians 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    earth_radius = 6367
    distance_km = earth_radius * c
    return distance_km

def get_distances_vectorized(trace, origin):
    distances = haversine_vectorized(
        trace['lat'], trace['lon'], origin['lat'], origin['lon']
    )
    return distances

def get_farthest_vectorized(trace, origin):
    distance = get_distances_vectorized(trace, origin)
    max_idx = distance.argmax()
    return trace.loc[max_idx], distance.loc[max_idx]

In [ ]:
%lprun -f haversine_vectorized get_farthest_vectorized(trace, origin)

In [ ]:
%%time
get_farthest_vectorized(trace, origin)

### 2.3 Exercise: Text Preprocessing

Preprocessing text is often a first step before analysis or modelling. There are few off-the-shelf text preprocessing pipelines that can meet every need, so we often find ourselves writing a set of functions to do the job for our specific task.

Here, we're going to examine a function that does some basic text cleaning on film reviews.

In [ ]:
!pip install spacy datasets
!python -m spacy download en_core_web_sm

In [ ]:
import spacy
import pandas as pd
from datasets import list_datasets, load_dataset

nlp = spacy.load("en_core_web_sm")
rotten = load_dataset('rotten_tomatoes')
rotten = rotten['train']['text'];

In [ ]:
rotten[:2]

In [ ]:
def tokens_to_string(tokens, max_length):
    """Convert a list of strings into a single space-separated string
    truncated to `max_length`.
    """
    text_string = ''
    n = 0
    for token in tokens:
        n += 1
        if n < max_length:
            text_string = text_string + token + ' '
        elif n == max_length:
            text_string = text_string + token

    return text_string
    
def make_spacy_docs(texts):
    """Generates spacy doc from a string."""
    for text in texts:
        yield nlp(text)


def clean_texts(texts, max_length=10):
    """Cleans and truncates a set of texts. Removes punctuation, stop
    words and numbers. Only returns the first `max_length` tokens.
    """
    docs = make_spacy_docs(texts)
    
    clean_texts = []
    for doc in docs:
        clean_tokens = []
        for token in doc:
            if token.is_punct:
                continue
            if token.is_stop:
                continue
            if token.is_digit:
                continue
            
            clean_tokens.append(token.text)

        clean_string = tokens_to_string(clean_tokens, max_length)
        clean_texts.append(clean_texts)

    return clean_texts

In [ ]:
%%time
_ = clean_texts(rotten[:100])

In [ ]:
%lprun -f clean_texts clean_texts(rotten[:100])

What are the bottlenecks in this code?

Hint: There is one big one, but you could squeeze even more performance from quite a few places.

#### Exercise
Speed up the text cleaning pipeline above using line profiler to help diagnose the bottlenecks.

Hints:
- Read the docs!
- There are many ways to solve this problem, but this notebook might not have all of the building blocks.

In [ ]:
# Write your text cleaning pipeline here:

def clean_texts_faster(texts, max_length):
    pass

In [ ]:
%%time
# time your code here

In [ ]:
%lprun # profile your code here

## 3. Future topics?

Discuss methods for speeding up your code.
- Caching results of functions that you might want to use again using [`lru_cache`](https://docs.python.org/3/library/functools.html#functools.lru_cache) 🗃
- Try out Cython 🐉
- Taking advantage of hardware but running code in parallel or on a GPU when you can't get any more speed out of your code or the gains would be huge 💪

## 4. [Draft] Make a Graph

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
blobs = make_blobs(1_000, centers=20, random_state=42)[0]

In [ ]:
plt.scatter(blobs[:, 0], blobs[:, 1])

In [ ]:
from scipy.spatial.distance import euclidean
from sklearn.metrics import pairwise_distances

In [ ]:
def get_knn(points, k):
    """"""
    pairwise_dists = pairwise_distances(points, metric='euclidean')
    knn = []
    for dists in pairwise_dists:
        knn_arg = np.argsort(dists)[1: k + 1]
        knn.append(knn_arg)
    return knn

def create_edgelist(knn):
    edges = []
    for s, targets in enumerate(knn):
        for t in targets:
            edges.append(sorted([s, t]))
    return edges

def calculate_dists():
    pass

In [ ]:
knn = get_knn(blobs, 5)
edges = create_edgelist(knn)